<a href="https://colab.research.google.com/github/sruthipsuresh/best-sellers-analysis/blob/main/NYTBL_analysis/Scripts/NYT_Bestseller_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
## NEW YORK TIMES SCRAPING
import os
import json
import time
import requests
import datetime
import dateutil
from dateutil.relativedelta import relativedelta
import pandas as pd

In [5]:
# Clean up genres and only query for those
genres = pd.read_csv("/content/NYTbookgenres.csv") #loading list data
#First remove columns which are remnants from json
del genres['status']
del genres['copyright']
del genres['num_results']
genres.head()
# As per genres.md, create a list of so-called "Stop words"
stop = ['Advice, How-To & Miscellaneous', 'Hardcover Graphic Books', 'Manga', 'Paperback Graphic Books', 'Children’s Chapter Books','Children’s Middle Grade', 'Children’s Middle Grade E-Book', 'Children’s Middle Grade Hardcover','Children’s Middle Grade Paperback','Children’s Paperback Books', 'Children’s Picture Books', 'Children’s Picture Books', 'Animals', 'Audio Fiction', 'Audio Nonfiction', 'Business', 'Crime and Punishment', 'Celebrities', 'Espionage', 'Expeditions', 'Fashion, Manners and Customs', 'Food and Diet','Games and Activities','Graphic Books and Manga','Health','Humor','Indigenous Americans','Love and Relationships','Mass Market','Parenthood and Family','Sports and Fitness','Travel']
for i in stop:
  genres = genres[genres['results__display_name'] != i]
filteredgenres = genres
filteredgenres.head()
filteredgenres.to_csv('filteredgenres.csv')

SyntaxError: ignored

In [57]:
# Now, to use NYT information and construct multiple datasets, and then join them.
# An example:
#r1 = pd.read_json('https://api.nytimes.com/svc/books/v3/lists/current/combined-print-and-e-book-fiction.json?api-key=1C9EFT4ikGhfCevT4RHR8GZqp6T9NAHm')
#r1 = r1.reindex(columns=['results']).iloc[1]
#r1 = r1.to_frame()

#with open('test.txt', 'w') as outfile:
 #   json.dump(r3, outfile)


ValueError: ignored

In [123]:
overall = requests.get('https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key=1C9EFT4ikGhfCevT4RHR8GZqp6T9NAHm').json()
with open('overall.txt', 'w') as outfile:
    json.dump(overall, outfile)
overallcsv = pd.read_csv("/content/overall.csv") #loading list data
print(overallcsv)

    status  ...              results__lists__books__buy_links__url
0       OK  ...  https://www.amazon.com/dp/059346527X?tag=NYTBS...
1      NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
2      NaN  ...  https://www.anrdoezrs.net/click-7990613-118195...
3      NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
4      NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
..     ...  ...                                                ...
535    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
536    NaN  ...  https://www.anrdoezrs.net/click-7990613-118195...
537    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
538    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
539    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...

[540 rows x 42 columns]


In [109]:
# Cleaning data, first obtain locations of the different lists.
# Cleaning data, first obtain locations of the different lists.
filter = overallcsv[overallcsv.results__lists__list_name.notnull()] # Filter to obtain indices
filter = filter.results__lists__list_name # Filter only one column
stop = ['Advice How-To & Miscellaneous', 'Hardcover Graphic Books', 'Manga', 'Paperback Graphic Books', 'Children’s Chapter Books','Children’s Middle Grade', 'Children’s Middle Grade E-Book', 'Childrens Middle Grade Hardcover','Childrens Middle Grade Paperback','Children’s Paperback Books', 'Children’s Picture Books', 'Picture Books', 'Animals', 'Audio Fiction', 'Audio Nonfiction', 'Business', 'Crime and Punishment', 'Celebrities', 'Espionage', 'Expeditions', 'Fashion, Manners and Customs', 'Food and Diet','Games and Activities','Graphic Books and Manga','Health','Humor','Indigenous Americans','Love and Relationships','Mass Market','Parenthood and Family','Sports and Fitness','Travel', 'Series Books', 'Middle Grade Paperback Monthly']
keep = pd.Series(filter)
for i in stop:
  keep = keep[keep != i]

print(keep)


0         Combined Print and E-Book Fiction
30     Combined Print and E-Book Nonfiction
60                        Hardcover Fiction
90                     Hardcover Nonfiction
120                 Trade Fiction Paperback
150                    Paperback Nonfiction
180         Advice How-To and Miscellaneous
300                   Young Adult Hardcover
390                          Business Books
450                     Mass Market Monthly
510           Young Adult Paperback Monthly
Name: results__lists__list_name, dtype: object


In [119]:
stop = set(filter) ^ set(keep) # Find differences in the keep and all genres list.
stop = pd.DataFrame(list(stop))
print(stop) # See the dropped books below.

                                  0
0    Middle Grade Paperback Monthly
1                  Audio Nonfiction
2                     Audio Fiction
3                     Picture Books
4  Childrens Middle Grade Hardcover
5           Graphic Books and Manga
6                      Series Books


In [120]:
print(filter)

0         Combined Print and E-Book Fiction
30     Combined Print and E-Book Nonfiction
60                        Hardcover Fiction
90                     Hardcover Nonfiction
120                 Trade Fiction Paperback
150                    Paperback Nonfiction
180         Advice How-To and Miscellaneous
210        Childrens Middle Grade Hardcover
240                           Picture Books
270                            Series Books
300                   Young Adult Hardcover
330                           Audio Fiction
360                        Audio Nonfiction
390                          Business Books
420                 Graphic Books and Manga
450                     Mass Market Monthly
480          Middle Grade Paperback Monthly
510           Young Adult Paperback Monthly
Name: results__lists__list_name, dtype: object


In [124]:
## DROP 210-299
## DROP 330-389, 420 - 449  abd 480-509
filteredbooks = overallcsv
filteredbooks = filteredbooks.drop(filteredbooks.index[210:299])
filteredbooks = filteredbooks.drop(filteredbooks.index[330:389])
filteredbooks = filteredbooks.drop(filteredbooks.index[420:449])
filteredbooks = filteredbooks.drop(filteredbooks.index[480:509])
print(filteredbooks)
filteredbooks.to_csv('filteredbooks.csv')

    status  ...              results__lists__books__buy_links__url
0       OK  ...  https://www.amazon.com/dp/059346527X?tag=NYTBS...
1      NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
2      NaN  ...  https://www.anrdoezrs.net/click-7990613-118195...
3      NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
4      NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
..     ...  ...                                                ...
535    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
536    NaN  ...  https://www.anrdoezrs.net/click-7990613-118195...
537    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
538    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...
539    NaN  ...  https://du-gae-books-dot-nyt-du-prd.appspot.co...

[392 rows x 42 columns]
